<a href="https://colab.research.google.com/github/mctrinh/ggColab/blob/main/starting_colab_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Colab to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!ls "/content/gdrive/My Drive/"   # reach the drive at anytime
!cd                               # bash commands start with !
                                  # !python script.py to run a python file
                                  # !git clone <REPOSITORY URL> to clone a repository

# Import resources

In [14]:
# Import resources
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import time
import json
import copy

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import PIL
# print(PIL.PILLOW_VERSION)

from PIL import Image
from collections import OrderedDict

import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
# !pip3 install torch torchvision
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

import os

# install library: !pip install <LIBRARY> or !apt-get install <LIBRARY>
# !pip install -q keras
import keras

# Upload and download files

In [ ]:
# upload files from computer
from google.colab import files
files.upload()

In [ ]:
# download files and run
from google.colab import files
files.download('/content/filename.pdf')

In [ ]:
# download and unzip a shared zip file link
# !wget <ENTER URL>
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!unzip -qq flower_data.zip

# GPUs in Colab

The default hardware in Colab is CPU. Go to *Runtime/Change runtime type* or *Edit/Notebook Settings* to enable GPU/TPU. When a GPU is connected, a maximum of 12 hours at a time on the Cloud Machine is given.
**To check if GPU is enable:**

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


**Observe TensorFlow speedup on GPU relative to CPU:**


In [6]:
%tensorflow_version 2.x 
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use GPU. Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu. \n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# Let's run each op(eration) once to warm up; refer: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op(eration) several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))


Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.6912122860003365
GPU (s):
0.04948854800022673
GPU speedup over CPU: 74x


# TPUs in Colab

Train a model to classify the images of flowers on Google's lightning-fast Cloud TPUs. The model will take as input a photo of a flower and return wether it is a daisy, dandelion, rose, sunflower, or tulip. We use the Keras framework, new to TPUs in TF 2.1.0.

Copyright 2019-2020 Google LLC. Licensed under the Apache License, Version 2.0 (the "License")

**Enabling and testing the TPU:**

# Utility

Colab automatically disconnect after 30 minutes idle. To stop this: F12 to open Chrome DevTools and add the following JavaScript snippet in the console. This makes a click on the connect-button every 60 seconds. Thus, Colab thinks that the notebook is not idle!



In [ ]:
# function KeepClicking(){
# console.log("Clicking");
# document.querySelector("colab-connect-button").click()
# }
# setInterval(KeepClicking,60000)